In [3]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
path = os.path.join(os.getcwd(), 'driftData')
labels = ['cover glass stack',]
fileNames = ['20151117-122926_CryoDrift4MinSteady.txt', 
             '20151117-123450_CryoDrift4MinSteady+10um.txt',
             '20151117-124428_CryoDrift4MinSteady+10um-A.txt',
             '20151117-124428_CryoDrift4MinSteady+10um-B.txt',
             '20151117-153125_CryoDrift4MinSteady+10um.txt']
colLabels = ['t','y','x','z']
colColours = [None, 'g', 'r', 'b']
skipRows = 1
skipCols = 1
fitPts = 4

Using matplotlib backend: MacOSX


In [2]:
pip install matplotlib


     |████████████████████████████████| 12.4 MB 9.2 MB/s eta 0:00:011
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
     |████████████████████████████████| 60 kB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 15.2 MB 5.2 MB/s eta 0:00:01
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.2/libexec/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
data = []
for fn in fileNames:
    fp = os.path.join(path, fn)
    d = np.genfromtxt(fp, skip_header=skipRows)[:,skipCols:]
    # Substract baselines.
    for col in range(1,d.shape[1]):
        d[:,col] -= d[:,col].min()
    data.append(d)
for d in data:
    for col in range(1, len(colLabels)):
        plt.plot(d[:,0], d[:,col], label=colLabels[col])


In [5]:
aligned = np.concatenate([d for d in data])
tOffsets = [290, 840, 5840, 10840]
for i in range(len(data)):
    if i == 0:
        count = data[i].shape[0]
        continue
    numPoints = data[i].shape[0]
    # Manual t offset.
    aligned[count:count+numPoints,0] += tOffsets[i-1]
    # Gradient
    deltaT = aligned[count-1, 0] - aligned[count, 0]
    for v in range(1,4):
        # Fit a stright line through a few points to estimate dvdt.
        dvdt = np.polyfit(aligned[count-fitPts-1:count-1, 0],
                      aligned[count-fitPts-1:count-1, v],
                      1)[0]   
        dv =  aligned[count,v]  + dvdt * deltaT - aligned[count-1, v]
        aligned[count:, v] -= dv
    count += numPoints   

    # Gradient by discrete differences in microns / time point
#gradient = np.diff(aligned, axis=0)
# Convert to nm / second
#gradient = 1000 * (gradient.T / gradient[:,0]).T

# Gradient by fitting in microns per second.
gradient = np.zeros(aligned.shape)
for col in [1,2,3]:
    for i in range(3, aligned.shape[0]-3):
        gradient[i, col] = np.polyfit(aligned[i-3:i+3, 0],
                                 aligned[i-3:i+3, col],
                                 1)[0]
# Convert to nm / second
gradient *= 1000.


In [6]:
fig = plt.figure()
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 3]) 
axl = fig.add_subplot(gs[1])
axl.xaxis.label.set_text('time / minutes')
axl.yaxis.label.set_text(u'displacement / µm')
axl.set_xticks(range(0, int(aligned.T[0].max()/60), 30))
axl.grid(b=True, which='major', alpha=0.2, linestyle='-')

axu = fig.add_subplot(gs[0])
axu.xaxis.set_visible(False)
axu.yaxis.label.set_text(r'drift / nm.s$^-1$')

handles = []
for col in [3,1,2]:
    handles.append(
            axl.plot(aligned.T[0]/60, aligned.T[col], label=colLabels[col],
                 ls=' ', marker='+', color=colColours[col]))
    axu.plot(aligned[:gradient.shape[0],0]/60, gradient[:,col],
             ls=' ', marker='.', markersize=2, color=colColours[col])

plt.title('Drift under cryo conditions')
#axl.legend(*axl.get_legend_handles_labels(), loc='upper left')


Text(0.5, 1.0, 'Drift under cryo conditions')

In [7]:
fig2=plt.figure()
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 3]) 
axl = fig2.add_subplot(gs[1])
axl.xaxis.label.set_text('time / minutes')
axl.yaxis.label.set_text(u'displacement / µm')
axl.set_xticks(range(0, int(aligned.T[0].max()/60), 30))
axl.grid(b=True, which='major', alpha=0.2, linestyle='-')

axu = fig2.add_subplot(gs[0])
axu.xaxis.set_visible(False)
axu.yaxis.label.set_text(r'drift / nm.s$^-1$')

axu.set_ylim([-3,5])

handles = []
for col in [3,1,2]:
    handles.append(
            axl.plot(aligned.T[0,900:1200]/60, 
                     aligned.T[col,900:1200]-aligned.T[col,900:1200].mean(), 
                     label=colLabels[col],
                  color=colColours[col]))
    axu.plot(aligned.T[0,900:1200]/60, gradient[900:1200,col],
             markersize=2, color=colColours[col])


plt.show()
        

In [36]:
gradient.shape

(1437, 4)

In [47]:
aligned.T[0,-1]

14410.0

In [ ]:
fig3=plt.figure()
h1= fig3.add_subplot()
h1.set_xlim([-3,3
             

In [ ]:
fig3=plt.figure()
h1= fig3.add_subplot()
h1.set_xlim([-3,3])
h1.xaxis.label.set_text('drift nm.axl.xaxis.label.set_text('time / minutes')

In [10]:
fig3=plt.figure()
h1= fig3.add_subplot()
h1.set_xlim([-3,3])
h1.xaxis.label.set_text('drift nm.s$^{-^1}$')
h1.yaxis.label.set_text('frequency')

for col in [1,2,3]:
    handles.append(
        h1.hist(gradient[:,col], bins=np.arange(-3,3,.1),color=colColours[col])
    )
plt.show()

In [66]:
bins=np.arange(-1,1,.1)
print (bins)

[-1.00000000e+00 -9.00000000e-01 -8.00000000e-01 -7.00000000e-01
 -6.00000000e-01 -5.00000000e-01 -4.00000000e-01 -3.00000000e-01
 -2.00000000e-01 -1.00000000e-01 -2.22044605e-16  1.00000000e-01
  2.00000000e-01  3.00000000e-01  4.00000000e-01  5.00000000e-01
  6.00000000e-01  7.00000000e-01  8.00000000e-01  9.00000000e-01]
